In [2]:
import re 
import numpy as np
import pandas as pd 
import matplotlib as mp 


# Danish CPR numbers consist of a six-digit date (DDMMYY), followed by a 4 digit identifier (IIII). Using the re module, the task is to write and apply a regular expression that matches any CPR number, either in the DDMMYYIIII format, or in the DDMMYY-IIII format. The regular expression should contain four groups, such that the DD, MM, YY, and IIII parts can be extracted after matching.

In [7]:
x= ''' # Measurements started
    Sep 9, 9:05, T=22deg
    SEP 9, 10:15, T=25deg
    # Taking a coffee break
    Sep 9, 11:15, T=-10deg
    # Weekend
    Sept 12, 09:00AM, T=32deg
    Oct12 13:00, T=32degr''' 


In [ ]:
print ( x)

 # Measurements started
    Sep 9, 9:05, T=22deg
    SEP 9, 10:15, T=25deg
    # Taking a coffee break
    Sep 9, 11:15, T=-10deg
    # Weekend
    Sept 12, 09:00AM, T=32deg
    Oct12 13:00, T=32degr


In [ ]:
x_lines= x.split(    '\n'    ) 
for x in   x_lines:
    if  x[  0  ] ==   '#  ' : 
     continue


print(x_line)



[' # Measurements started', '    Sep 9, 9:05, T=22deg', '    SEP 9, 10:15, T=25deg', '    # Taking a coffee break', '    Sep 9, 11:15, T=-10deg', '    # Weekend', '    Sept 12, 09:00AM, T=32deg', '    Oct12 13:00, T=32degr']


[' # Measurements started',
 '    Sep 9, 9:05, T=22deg',
 '    SEP 9, 10:15, T=25deg',
 '    # Taking a coffee break',
 '    Sep 9, 11:15, T=-10deg',
 '    # Weekend',
 '    Sept 12, 09:00AM, T=32deg',
 '    Oct12 13:00, T=32degr']

In [ ]:

# Create regular expression

             		

